<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/oxford_IIIT_pet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install segmentation-models-pytorch
!pip install albumentations

In [14]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive/Colab Notebooks/[CV] Oxford-IIIT Pet"

!cp "{drive_path}/images.tar.gz" .
!cp "{drive_path}/annotations.tar.gz" .

!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
    A.Resize(128, 128),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# [검증/테스트용] 오직 크기 조절과 정규화만 (시험 문제 오염 방지)
val_transform = A.Compose([
    A.Resize(128, 128),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [16]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import os

class PetDataset(Dataset):
    def __init__(self, image_dir, mask_dir, file_list, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        # 1. 파일 경로 설정
        img_name = self.file_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png'))

        # 2. 이미지와 마스크 읽기 (넘파이 배열로 유지)
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path))

        # 3. 변환(Augmentation + Resize + Normalize + ToTensor) 적용
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # 4. 마스크 정수형 변환 및 값 조정 (1,2,3 -> 0,1,2)
        # 만약 transform에서 ToTensorV2를 썼다면 이미 텐서일 수 있습니다.
        if not isinstance(mask, torch.Tensor):
            mask = torch.from_numpy(mask).long()

        mask = mask.long() - 1
        mask = torch.clamp(mask, min=0, max=2)

        return image, mask

# --- 사용 예시 ---
# train_dataset = PetDataset("images/", "annotations/trimaps/", train_files, transform=train_transform)

In [17]:
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

all_files = sorted([f for f in os.listdir("images/") if f.endswith('.jpg')])

# 2. 파일명 리스트를 8:2 비율로 나눕니다. (random_state는 결과 고정을 위해 설정)
# 이렇게 파일명 자체를 나눠버리면 절대로 인덱스가 겹칠 일이 없습니다.
train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)

# 2. 각각 다른 변환을 적용해서 생성
train_dataset = PetDataset(image_dir="images/", mask_dir="annotations/trimaps/", file_list=train_files, transform=train_transform)
val_dataset = PetDataset(image_dir="images/", mask_dir="annotations/trimaps/", file_list=val_files, transform=val_transform)

# 3. 각각의 DataLoader 만들기
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"학습 데이터: {len(train_dataset)}개")
print(f"검증 데이터: {len(val_dataset)}개")

학습 데이터: 5912개
검증 데이터: 1478개


In [18]:
# 1. 데이터셋에서 샘플 하나 가져오기
sample_img, sample_mask = train_dataset[0]

# 2. 마스크의 기본 정보 출력
print(f"마스크 타입: {type(sample_mask)}")
print(f"마스크 모양 (Shape): {sample_mask.shape}") # [128, 128]이어야 함

# 3. 마스크에 들어있는 실제 값 종류 확인 (Unique values)
# 여기서 0, 1, 2 외의 값(-1, 3, 255 등)이 나오면 에러의 주범입니다!
unique_values = torch.unique(sample_mask)
print(f"마스크 내 포함된 값 종류: {unique_values.tolist()}")

# 4. 혹시 값이 너무 많다면 개수도 확인
for val in unique_values:
    count = (sample_mask == val).sum()
    print(f"값 {val.item()}: {count.item()} 픽셀")

마스크 타입: <class 'torch.Tensor'>
마스크 모양 (Shape): torch.Size([128, 128])
마스크 내 포함된 값 종류: [0, 1, 2]
값 0: 1763 픽셀
값 1: 13515 픽셀
값 2: 1106 픽셀


In [19]:
import segmentation_models_pytorch as smp

# 드롭아웃 비율 설정 (0.2~0.3 정도가 무난합니다)
aux_params = dict(
    pooling='avg',             # 풀링 방식
    dropout=0.3,               # 드롭아웃 비율
    activation=None,      # 활성화 함수
    classes=3,                 # 클래스 수
)

model = smp.Unet(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3,
    aux_params=aux_params      # 여기서 드롭아웃이 적용됩니다!
)

# 과적합 방지를 위해 드롭아웃을 추가하고 싶다면?
# 이 라이브러리는 디코더 부분에 드롭아웃 비율을 설정할 수 있습니다.
# model = smp.Unet(..., decoder_use_batchnorm=True) 처럼 기본 설정이 잘 되어 있지만,
# 직접 레이어를 수정하기보다 하이퍼파라미터로 조절하는 것이 일반적입니다.

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Unet(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_expand_conv): Identity()
        (_bn0): Identity()
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identit

In [20]:
import torch.optim as optim
import torch.nn as nn
import segmentation_models_pytorch as smp

# 1. Dice Loss와 CrossEntropy를 섞은 복합 손실함수 선언
# smp 라이브러리에 이미 잘 구현되어 있습니다.
criterion = smp.losses.DiceLoss(mode='multiclass') # 영역 기반
ce_loss = nn.CrossEntropyLoss() # 픽셀 기반

# 2. 학습 루프에서 사용할 때는 이렇게 섞어줍니다.
def hybrid_loss(outputs, masks):
    return ce_loss(outputs, masks) + criterion(outputs, masks)

# 2. 최적화 도구: 가장 무난하고 성능 좋은 Adam
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [21]:
from tqdm.auto import tqdm # 학습 진행 상황을 바로 보여주는 라이브러리

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train() # 모델을 학습 모드로 설정 (드롭아웃 등이 활성화됨)
    running_loss = 0.0

    # tqdm으로 진행바 표시
    pbar = tqdm(loader, desc="Training")

    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)

        # 1. 변화도(Gradient) 초기화
        optimizer.zero_grad()

        outputs = model(images)

# 만약 결과가 튜플(또는 리스트)로 오면 첫 번째 것만 선택
        if isinstance(outputs, (list, tuple)):
          outputs = outputs[0]

        # 3. 손실(Loss) 계산
        loss = criterion(outputs, masks)

        # 4. 역전파 (Backward) - 오차를 뒤로 전달
        loss.backward()

        # 5. 가중치 업데이트 (Step) - 모델 수정
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    return running_loss / len(loader)

In [22]:
def validate(model, loader, criterion, device):
    model.eval() # ⚠️ 중요: 평가 모드 (드롭아웃 비활성화)
    val_loss = 0.0

    with torch.no_grad(): # ⚠️ 중요: 가중치 업데이트 안 함 (메모리 절약)
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            if isinstance(outputs, (list, tuple)):
              outputs = outputs[0]
            loss = criterion(outputs, masks)
            val_loss += loss.item()

    return val_loss / len(loader)

In [23]:
# 에포크 수 설정
epochs = 10
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

for epoch in range(epochs):
    # 1. 학습
    train_loss = train_one_epoch(model, train_loader, optimizer, hybrid_loss, device)

    # 2. 검증
    val_loss = validate(model, val_loader, hybrid_loss, device)
    scheduler.step()

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # 만약 Val Loss가 낮아졌다면 모델을 드라이브에 저장 (아까 배운 !cp 활용)
    # torch.save(model.state_dict(), "best_model.pth")
    # !cp best_model.pth "{drive_path}/"

Training:   0%|          | 0/370 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 0.8666 | Val Loss: 0.5192


Training:   0%|          | 0/370 [00:00<?, ?it/s]

Epoch 2 | Train Loss: 0.5228 | Val Loss: 0.4523


Training:   0%|          | 0/370 [00:00<?, ?it/s]

...

Epoch 8 | Train Loss: 0.3881 | Val Loss: 0.3969


Training:   0%|          | 0/370 [00:00<?, ?it/s]

Epoch 9 | Train Loss: 0.3845 | Val Loss: 0.3973


Training:   0%|          | 0/370 [00:00<?, ?it/s]

Epoch 10 | Train Loss: 0.3812 | Val Loss: 0.3978
